# <center>Pet Dog Behavior Study</center>
<center>Dinwoodie, Dodman, et al.</center>
<center>Tufts University</center>

In [6]:
import sqlite3
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
from matplotlib_venn import venn2, venn3
import scipy.stats as scs
import textwrap
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from  itertools import combinations

# IPython magics for this notebook.
%matplotlib inline

# Initiate plotly IPython notebook mode.
py.init_notebook_mode(connected=True)

In [7]:
class FieldRegistry:
    
    def __init__(self):
        self.fields = {}
        self.labels = {}
    
    def addToRegistry(self, index, labels, fields, category):
        self.labels[category] = {}
        for counter, value in enumerate(labels, 1):
            key = '{}{:02}'.format(index, counter)
            self.labels[category][key] = value
        self.fields[category] = fields
        
FR = FieldRegistry()   
CATEGORY = ['Aggression', 'Fear/Anxiety', 'Compulsion', 'House Soiling', 'Excessive Barking', 
            'Jumping', 'Mounting/Humping', 'Consuming Feces', 'Destructive Behavior',
            'Rolling in Repulsive Material', 'Running Away/Escaping', 
            'Overactivity/Hyperactivity']
        
# Aggression.
labels = ['Familiar people in the home', 'Stangers visiting the home',
          'Stangers away from the home', 'Another dog in the home',
          'Unfamiliar dogs visiting the home', 'Unfamiliar dogs on walks (off lead)',
          'Unfamiliar dogs on walks (on lead)', 'Veterinarians', 'Trainers', 'Groomers',
          'Animals other than dogs in the home']
fields = ['q03_main_1', 'q03_main_2', 'q03_main_3', 'q03_main_4', 'q03_main_5', 'q03_main_6',
          'q03_main_7', 'q03_main_8', 'q03_main_9', 'q03_main_10', 'q03_main_11']
FR.addToRegistry('A', labels, fields, CATEGORY[0])

# Fear/anxiety.
labels = ['Thunderstorm phobia', 'Noise phobia', 'Crowd phobia', 'Phobia of other dogs',
          'PTSD', 'Generalized anxiety', 'Situational anxiety', 'Veterinarian phobia',
          'Separation anxiety', 'Travel anxiety', 'Other']
fields = ['q04_1', 'q04_2', 'q04_3', 'q04_4', 'q04_5', 'q04_6', 'q04_7', 'q04_8', 'q04_9',
          'q04_10', 'q04_11']
FR.addToRegistry('B', labels, fields, CATEGORY[1])

# Compulsion.
labels = ['Spinning', 'Tail chasing', 'Shadow/light chasing', 'Running in geometric patterns',
          'Licking of wrist/hock', 'Fly snapping', 'Sucking flank region/blankets',
          'Tennis ball fetish', 'Collecting/arranging objects', 'Nail biting',
          'Digging in yard', 'Stone/rock chewing', 'Other']
fields = ['q05_main_1', 'q05_main_2', 'q05_main_3', 'q05_main_4', 'q05_main_5', 'q05_main_6',
          'q05_main_7', 'q05_main_8', 'q05_main_9', 'q05_main_10', 'q05_main_11',
          'q05_main_12', 'q05_main_13']
FR.addToRegistry('C', labels, fields, CATEGORY[2])

# House soiling.
## soil_type = ['Urine', 'Feces', 'Urine and feces']
labels = ['Urine', 'Feces', 'Urine and feces', 'Specific locations', 'Anywhere',
          'Owner present', 'Owner away', 'Excited/overwhelmed']
fields = ['q06_soil_type_1', 'q06_soil_type_2', 'q06_soil_type_3','q06_soil_location_1',
          'q06_soil_location_2', 'q06_situation_1', 'q06_situation_2', 'q06_situation_3']
FR.addToRegistry('D', labels, fields, CATEGORY[3])

# Excessive barking.
labels = ['Owner present', 'Owner away', 'To get attention', 'At tiggers (inside)',
          'At triggers (outside)', 'During car rides']
fields = ['q07_sitatuon_1', 'q07_sitatuon_2', 'q07_sitatuon_3', 'q07_sitatuon_4',
          'q07_sitatuon_5', 'q07_sitatuon_6']
FR.addToRegistry('E', labels, fields, CATEGORY[4])

# Jumping.
labels = ['Owner', 'Family members', 'Strangers']
fields = ['q08_who_1', 'q08_who_2', 'q08_who_3']
FR.addToRegistry('F', labels, fields, CATEGORY[5])

# Mounting/humping.
labels = ['People', 'Familiar dogs', 'Unfamiliar dogs', 'Inanimate objects']
fields = ['q09_main_1', 'q09_main_2', 'q09_main_3', 'q09_main_4']
FR.addToRegistry('G', labels, fields, CATEGORY[6])

# Consuming feces.
labels = ['Their own', "Other dogs'", "Other species'"]
fields = ['q10_main_1', 'q10_main_2', 'q10_main_3']
FR.addToRegistry('H', labels, fields, CATEGORY[7])

# Destructive behavior.
labels = ['Owner is home', 'Owner is away']
fields = ['q11_situation_1', 'q11_situation_2']
FR.addToRegistry('H', labels, fields, CATEGORY[8])

# Rolling in repulsive materials.
labels = ['Urine', 'Feces', 'Dead Stuff', 'Garbage']
fields = ['q12_main_1', 'q12_main_2', 'q12_main_3', 'q12_main_4']
FR.addToRegistry('J', labels, fields, CATEGORY[9])

# Running away/escaping.
labels = ['Escapes when out', 'Escapes from home', 'Escapes from confinement',
          'Returns home after escape']
fields = ['q14_out', 'q14_house', 'q14_conf', 'q14_return']
FR.addToRegistry('K', labels, fields, CATEGORY[10])

# Overactivity/hyperactivity.
labels = ['Constant moving/jumping', 'Difficulty settling', 'Highly distractible',
          'Impulsive']
fields = ['q15_main_1', 'q15_main_2', 'q15_main_3', 'q15_main_4']
FR.addToRegistry('L', labels, fields, CATEGORY[11])

In [8]:
# Data Globals
SIG_P = 0.01
TOTAL_USERS = 0
TOTAL_DOGS = 0
REMAINING_DOGS = 0
PREVALENCE = lambda x: (x / REMAINING_DOGS) * 100
CHI2_MATRIX = pd.DataFrame()

# Database Globals
USER_TABLE = 'users'
DOG_TABLE = 'dogs'
BIAS_FILTER = '''
    JOIN users
    USING (record_id)
    WHERE question_reason_for_part_3 = 0
    OR (question_reason_for_part_3 = 1 AND q01_main != 1)'''
CON = sqlite3.connect('../data/processed/processed.db')

In [9]:
def createStringDataFrame(table, fields, labels, filtered):
    query = 'SELECT ' + fields + ' FROM ' + table
    if filtered:
        query += ' ' + BIAS_FILTER
    df = pd.read_sql_query(query, CON)
    df.columns = labels
    return df

def convertToNumeric(df):
    for col in df:
        df[col] = pd.to_numeric(df[col])
    return df

def createNumericDataFrame(table, fields, labels, filtered):
    df = createStringDataFrame(table, fields, labels, filtered)
    return convertToNumeric(df)

def replaceFields(df, replacement_dict):
    for key, value in replacement_dict.items():
        df = df.replace(key, value)
    return df

def getValueCountAndPrevalence(df, field):
    s = df[field].value_counts()
    p = s.apply(PREVALENCE)
    rv = pd.concat([s, p], axis=1)
    rv.columns = ['sum', 'prevalence']
    return rv
    
def createChiSquaredMatrix():
    fields = ''
    for cat, sublist in FR.fields.items():
        for field in sublist:
            fields += '{}, '.format(field)
    fields = fields[:-2]
    labels = []
    for cat, subdict in FR.labels.items():
        for key, value in  subdict.items():
            labels.append(key)
    print(fields)
    print(labels)
    df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
    cols = []
    chi2 = {}
    for col in df:
        cols.append(col)
        chi2[col] = {}
    pairs = list(combinations(df.columns,2))
    for pair in pairs:
        contingency = pd.crosstab(df[pair[0]], df[pair[1]])
        c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
        chi2[pair[0]][pair[1]] = c
        chi2[pair[1]][pair[0]] = c
    df = pd.DataFrame(chi2).sort_index(ascending=True)
    return df

def checkSignificance(p, sig=SIG_P):
    if p < sig:
        print('The resulting p-value is below the set significance threshold (%.2f).' %sig)

In [10]:
if CHI2_MATRIX.empty:
    CHI2_MATRIX = createChiSquaredMatrix()

q03_main_1, q03_main_2, q03_main_3, q03_main_4, q03_main_5, q03_main_6, q03_main_7, q03_main_8, q03_main_9, q03_main_10, q03_main_11, q04_1, q04_2, q04_3, q04_4, q04_5, q04_6, q04_7, q04_8, q04_9, q04_10, q04_11, q05_main_1, q05_main_2, q05_main_3, q05_main_4, q05_main_5, q05_main_6, q05_main_7, q05_main_8, q05_main_9, q05_main_10, q05_main_11, q05_main_12, q05_main_13, q06_soil_type_1, q06_soil_type_2, q06_soil_type_3, q06_soil_location_1, q06_soil_location_2, q06_situation_1, q06_situation_2, q06_situation_3, q07_sitatuon_1, q07_sitatuon_2, q07_sitatuon_3, q07_sitatuon_4, q07_sitatuon_5, q07_sitatuon_6, q08_who_1, q08_who_2, q08_who_3, q09_main_1, q09_main_2, q09_main_3, q09_main_4, q10_main_1, q10_main_2, q10_main_3, q11_situation_1, q11_situation_2, q12_main_1, q12_main_2, q12_main_3, q12_main_4, q14_out, q14_house, q14_conf, q14_return, q15_main_1, q15_main_2, q15_main_3, q15_main_4
['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'B01', 'B02', 'B03', 

DatabaseError: Execution failed on sql 'SELECT q03_main_1, q03_main_2, q03_main_3, q03_main_4, q03_main_5, q03_main_6, q03_main_7, q03_main_8, q03_main_9, q03_main_10, q03_main_11, q04_1, q04_2, q04_3, q04_4, q04_5, q04_6, q04_7, q04_8, q04_9, q04_10, q04_11, q05_main_1, q05_main_2, q05_main_3, q05_main_4, q05_main_5, q05_main_6, q05_main_7, q05_main_8, q05_main_9, q05_main_10, q05_main_11, q05_main_12, q05_main_13, q06_soil_type_1, q06_soil_type_2, q06_soil_type_3, q06_soil_location_1, q06_soil_location_2, q06_situation_1, q06_situation_2, q06_situation_3, q07_sitatuon_1, q07_sitatuon_2, q07_sitatuon_3, q07_sitatuon_4, q07_sitatuon_5, q07_sitatuon_6, q08_who_1, q08_who_2, q08_who_3, q09_main_1, q09_main_2, q09_main_3, q09_main_4, q10_main_1, q10_main_2, q10_main_3, q11_situation_1, q11_situation_2, q12_main_1, q12_main_2, q12_main_3, q12_main_4, q14_out, q14_house, q14_conf, q14_return, q15_main_1, q15_main_2, q15_main_3, q15_main_4 FROM dogs 
    JOIN users
    USING (record_id)
    

## 1. Participant Demographics

### 1.1 Number of Participants:

In [ ]:
df = createNumericDataFrame(USER_TABLE, 'COUNT(*)', ['count'], filtered=False)
# Assign value to global.
TOTAL_USERS = df['count'][0]
print('Total number of participants: %d' %TOTAL_USERS)

### 1.2 Motivation for Enrollment:

In [ ]:
# Create the necessary dataframe.
fields = ('question_reason_for_part_1, question_reason_for_part_2, '
          'question_reason_for_part_3, question_reason_for_part_4, '
          'question_reason_for_part_5')
labels = ['Love for dogs', 'You help shelter animals', 'Suspicion of behavior problems',
          'Work with animals', 'Other']
df = createNumericDataFrame(USER_TABLE, fields, labels, filtered=False)
    
# Create a bar graph to illustrate breakdown of responses.
sums = df.sum()
sums = sums.sort_values(ascending=True)
data = [go.Bar(y=sums.index.values,
               x=sums.values,
               orientation='h')]
layout = go.Layout(title="Motivation for Study Enrollment",
                   xaxis=dict(title='number of participants'),
                   yaxis=dict(title='',
                              automargin=True,
                              tickangle=30,
                              tickprefix=' ',
                              ticksuffix=' '),
                   width=600,
                   height=400)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

# NOTE: This does not include a count of users who did not provide a reason.

## 2 Participating Dog Demographics

### 2.1 Number of Participating Dogs:

In [ ]:
df = createNumericDataFrame(DOG_TABLE, 'COUNT(*)', ['count'], filtered=False)
# Assign value to global.
TOTAL_DOGS = df['count'][0]
print('Total number of participating dogs: %d' %TOTAL_DOGS)

### 2.2 Adjusting Sample for Bias

It is in this section that we define and apply a filter to remove bias from the data set.
 
The current bias filter focuses largely on the participants' motivation for enrollment ("Why did you decide to participate in this survey?", pg. 1). The answer choice "my dog may have a behavior problem" was included to identify participants who enrolled with motivational bias. However, participants may not have a motiivational bias towards all of their participating dogs. To account for this, the question "do you believe \[dog name\] has ever had a behavior problem" (pg. 4) was used in conjunction with the enrollment motivation to identify the dogs most likely to be influenced by the bias of the participating owner. 

In [ ]:
fields = 'q02_score'
labels = ['Score']

df_gross = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=False)
cnt_total_dogs_w_problems = len(df_gross[df_gross[labels[0]] != 0].index)

df_adjusted = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
REMAINING_DOGS = len(df_adjusted.index)
cnt_total_dogs_w_problems_adjusted = len(df_adjusted[df_adjusted[labels[0]] != 0].index)

# Display the count results.
print('Pre-adjustment Responses (Gross):')
print('Total number of dogs: %d' %TOTAL_DOGS)
print('Total number of dogs with a reported behavior problem: %d' %cnt_total_dogs_w_problems)

print('\nPost-adjustment Responses (Adjusted):')
print('Total number of dogs: %d' %REMAINING_DOGS)
print('Total number of dogs with a reported behavior problem: %d'
      %cnt_total_dogs_w_problems_adjusted)

### 2.3 Prevalence of Overall Behavior Problems

Prevalence is defined as the proportion of a population found to be affected by a medical condition. This section aims to calculate the aggregate prevalence of the most common behavior problems in a given canine population.

In [ ]:
# Calculate the gross prevalence.
prevalence_gross = (cnt_total_dogs_w_problems / TOTAL_DOGS) * 100

# Calculate the adjusted prevalence.
prevalence_adjusted = PREVALENCE(cnt_total_dogs_w_problems_adjusted)

# Calculate the difference between the gross and adjusted prevalence.
diff_prevalence = prevalence_adjusted - prevalence_gross
sign = '+' if diff_prevalence > 0 else ''

# Display the prevalence results.
print('Pre-adjustment prevalence: %.2f%% (%d/%d dogs)'
      %(prevalence_gross, cnt_total_dogs_w_problems, TOTAL_DOGS))
print('Post-adjustment prevalence: %.2f%% (%d/%d dogs)'
      %(prevalence_adjusted, cnt_total_dogs_w_problems_adjusted, REMAINING_DOGS))
print('Shift in prevalence as a result of adjustment: %s%.2f%%'
      %(sign, diff_prevalence))

### 2.2 Gender:

In [ ]:
# Construct the dataframe.
fields = 'dog_sex'
labels = ['Gender']
df = createStringDataFrame(DOG_TABLE, fields, labels, filtered=True)
replacements = {'':'No response', '1':'Male', '2':'Female'}
df = replaceFields(df, replacements)

# Calculate sums and prevalences.
calc = getValueCountAndPrevalence(df, labels[0])

# Display the results as a table.
data = [go.Table(header=dict(values=[labels[0], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[calc.index.values, calc['sum'].values, 
                                    calc['prevalence'].values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Gender of Participating Dogs',
                   width=700,
                   height=200,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 2.3 Neutered Status:

In [ ]:
# Construct the dataframe.
fields = 'dog_spayed'
labels = ['Neutered']
df = createStringDataFrame(DOG_TABLE, fields, labels, filtered=True)
replacements = {'':'No response', '0':'No', '1':'Yes', '2':"I don't know"}
df = replaceFields(df, replacements)

# Calculate sums and prevalences.
calc = getValueCountAndPrevalence(df, labels[0])

data = [go.Table(header=dict(values=[labels[0], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[calc.index.values, calc['sum'].values, 
                                    calc['prevalence'].values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Neutered Status of Participating Dogs',
                   width=500,
                   height=225,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 2.4 Breakdown of Participating Purebred Breeds:

In [ ]:
# Create the necessary dataframe.
fields = 'purebred_breed'
labels = ['Breed']
df = createStringDataFrame(DOG_TABLE, fields, labels, filtered=False)

df = df.replace('', np.nan, regex=True)
df = df.replace('118', 'Labrador retriever', regex=True)
df = df.replace('92', 'German shepherd', regex=True)
df = df.replace('97', 'Golden retriever', regex=True)
df = df.replace('151', 'Poodle', regex=True)
df = df.replace('43', 'Border collie', regex=True)
df = df.replace('71', 'Cocker spaniel', regex=True)
df = df.replace('16', 'Australian shepherd', regex=True)
df = df.replace('75', 'Dachshund', regex=True)
df = df.replace('64', 'Chihuahua', regex=True)
df = df.replace('170', 'Shetland sheepdog', regex=True)

# Calculate sums and prevalences.
calc = getValueCountAndPrevalence(df, labels[0])
calc = calc[:10]

data = [go.Table(columnwidth = [100, 100, 100],
                 header=dict(values=[labels[0], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[calc.index.values, calc['sum'].values, 
                                    calc['prevalence'].values],

                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Top 10 Participating Breeds',
                   width=700,
                   height=350,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

## 4 Prevalence of Individual Behavior Problems

This section aims to calculate the prevalence for each behavior problem included in the study. The adjusted dataset is utilized to eliminate potential bias.

In [ ]:
# Create the necessary dataframe.
fields = []
labels = []
for counter, category in enumerate(CATEGORY, 1):
    if counter > 10:
        counter += 1;
    fields.append('q02_main_{}'.format(counter))
    labels.append(category)
fields = ', '.join(fields)
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
total_dogs = len(df.index)
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(header=dict(values=['Behavior Problem', 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of Individual Behavior Problems',
                   width=700,
                   height=375,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

In [ ]:
prevalences = prevalences.sort_values(ascending=True)
data = [go.Bar(x=prevalences.values,
               y=prevalences.index.values,
               orientation='h')]
layout = go.Layout(title='Prevalence of Individual Behavior Problems',
                   xaxis=dict(title='prevalence (%)'),
                   yaxis=dict(title='', automargin=True, tickangle=30, tickprefix=' ',
                              ticksuffix=' '),
                   width=700,
                   height=500)
fig = dict(data=data, layout=layout)
py.iplot(fig)

## 5 Behavior Problem Compositions

### 5.1 Aggression:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[0]])
labels = list(FR.labels[CATEGORY[0]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[0], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[0]),
                   width=700,
                   height=350,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

In [ ]:
# Generate a bar graph.
prevalences = prevalences.sort_values(ascending=True)
data = [go.Bar(x=prevalences.values,
               y=prevalences.index.values,
               orientation='h')]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[0]),
                   xaxis=dict(title='prevalence (%)'),
                   yaxis=dict(title='', automargin=True, tickangle=30, tickprefix=' ',
                              ticksuffix=' '),
                   width=700,
                   height=500
                  )
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 5.2 Fear/Anxiety:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[1]])
labels = list(FR.labels[CATEGORY[1]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[1], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[1]),
                   width=700,
                   height=350,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

In [ ]:
# Generate a bar graph.
prevalences = prevalences.sort_values(ascending=True)
data = [go.Bar(x=prevalences.values,
               y=prevalences.index.values,
               orientation='h')]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[1]),
                   xaxis=dict(title='prevalence (%)'),
                   yaxis=dict(title='', automargin=True, tickangle=30, tickprefix=' ',
                              ticksuffix=' '),
                   width=700,
                   height=500)
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 5.3 Compulsion:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[2]])
labels = list(FR.labels[CATEGORY[2]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[2], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[2]),
                   width=700,
                   height=400,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

In [ ]:
# Generate a bar graph.
prevalences = prevalences.sort_values(ascending=True)
data = [go.Bar(x=prevalences.values,
               y=prevalences.index.values,
               orientation='h')]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[2]),
                   xaxis=dict(title='prevalence (%)'),
                   yaxis=dict(title='', automargin=True, tickangle=30, tickprefix=' ',
                              ticksuffix=' '),
                   width=700,
                   height=500)
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 5.4 House Soiling:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[3]])
labels = list(FR.labels[CATEGORY[3]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[3], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[3]),
                   width=700,
                   height=300,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 5.5 Excessive Barking:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[4]])
labels = list(FR.labels[CATEGORY[4]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[4], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[4]),
                   width=700,
                   height=250,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 5.6 Jumping:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[5]])
labels = list(FR.labels[CATEGORY[5]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[5], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[5]),
                   width=700,
                   height=200,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 5.7 Mounting/Humping:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[6]])
labels = list(FR.labels[CATEGORY[6]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[6], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[6]),
                   width=700,
                   height=225,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 5.8 Consumption of Feces:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[7]])
labels = list(FR.labels[CATEGORY[7]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[7], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[7]),
                   width=700,
                   height=200,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 5.9 Destructive Behavior:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[8]])
labels = list(FR.labels[CATEGORY[8]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[8], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[8]),
                   width=700,
                   height=175,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 5.10 Rolling in Repulsive Materials:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[9]])
labels = list(FR.labels[CATEGORY[9]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[9], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[9]),
                   width=700,
                   height=225,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 5.11 Running Away/Escaping:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[10]])
labels = list(FR.labels[CATEGORY[10]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[10], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[10]),
                   width=700,
                   height=225,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 5.12 Overactivity/Hyperactivity:

In [ ]:
# Create the necessary dataframe.
fields = ', '.join(FR.fields[CATEGORY[11]])
labels = list(FR.labels[CATEGORY[11]].values())
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Get individual behavior problem counts and display as a table.
sums = df.sum()
sums = sums.sort_values(ascending=False)

# Calculate the prevalence of each behavior problem.
prevalences = sums.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 100],
                 header=dict(values=[CATEGORY[11], 'Number of Dogs', 'Prevalence (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[sums.index.values, sums.values, prevalences.values],
                            align = ['left'] * 3,
                            format = [None, 'd', '.2f']))]
layout = go.Layout(title='Prevalence of {}'.format(CATEGORY[11]),
                   width=700,
                   height=225,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

## 6 Separation Anxiety, Noise Phobia, and Thunderstorm Phobia

### 6.1 Individual Independence:

In [ ]:
# Create the necessary dataframe.
fields = 'q04_1, q04_2, q04_9'
labels = ['thunderstorm phobia', 'noise phobia', 'separation anxiety']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

pairs = [['thunderstorm phobia', 'noise phobia'],
         ['thunderstorm phobia', 'separation anxiety'],
         ['noise phobia', 'separation anxiety']]

titles = ['Thunderstorm Phobia and Noise Phobia',
          'Thunderstorm Phobia and Separation Anxiety',
          'Noise Phobia and Separation Anxiety']

counter = 0
for pair in pairs:
    # Create a contingency table.
    contingency = pd.crosstab(df[pair[0]], df[pair[1]])
    # Convert the cross tabulated dataframe to a series of sums.
    d = {pair[0]: contingency[0][1],
         pair[1]: contingency[1][0],
         'both': contingency[1][1]}   
    cross_sums = pd.Series(d)
    # Display the cross tabulated data as a venn diagram.
    labels = cross_sums.index.values
    venn2(cross_sums, set_labels=labels[0:2])
    plt.title(titles[counter])
    plt.show()
    # Execute a chi-squared test of independence.
    print('Chi-squared Test of Independence for %s and %s:' %(pair[0], pair[1]))
    c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
    print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
    checkSignificance(p)
    counter += 1

### 6.2 Grouped Independence:

In [ ]:
# Create a contingency table.
contingency = pd.crosstab(df['separation anxiety'], [df['noise phobia'],
                                                     df['thunderstorm phobia']])

# Convert the cross tabulated dataframe to a series of sums.
d = {'separation anxiety': contingency[0][0][1],
     'noise phobia': contingency[1][0][0],
     'separation-noise': contingency[1][0][1],
     'thunderstorm phobia': contingency[0][1][0],
     'separation-thunderstorm': contingency[0][1][1],
     'noise-thunderstorm': contingency[1][1][0],     
     'all': contingency[1][1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
labels = [cross_sums.index.values[i] for i in [0, 1, 3]]
venn3(cross_sums, set_labels=labels)
plt.title('Separation Anxiety, Noise Phobia, and Thunderstorm Phobia')
plt.show()

# Execute a chi-squared test of independence.
title = ('Chi-squared Test of Independence for separation anxiety and the combination of '
         'noise and thunderstorm phobia:')
print(textwrap.fill(title, width=90))
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)

## 7 Compulsion and Fearful/Anxious Behaviors

In [ ]:
# Create the necessary hdataframe.
fields = 'q02_main_2, q02_main_3'
labels = ['fear/anxiety', 'compulsion']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

# Cross tabulate the relevant columns.
contingency = pd.crosstab(df['fear/anxiety'], df['compulsion'])

# Convert the cross tabulated dataframe to a series of sums.
d = {'fear/anxiety': contingency[0][1],
     'compulsion': contingency[1][0],
     'both': contingency[1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
labels = cross_sums.index.values
venn2(cross_sums, set_labels=labels[0:2])
plt.title('Fear/Anxiety and Compulsion')
plt.show()

In [ ]:
# Execute a chi-squared test of independence.
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('Chi-square Test of Independence:')
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)

## 8 Aggression and Fearful/Anxious Behavior

### 8.1 Overall Aggression and Fearful/Anxious Behavior:

In [ ]:
# Create the necessary dataframe.
fields = 'q02_main_1, q02_main_2'
labels = ['aggression', 'fear/anxiety']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
    
# Create a contingency table.
contingency = pd.crosstab(df['aggression'], df['fear/anxiety'])

# Convert the cross tabulated dataframe to a series of sums.
d = {'aggression': contingency[0][1],
     'fear/anxiety': contingency[1][0],
     'both': contingency[1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
labels = cross_sums.index.values
venn2(cross_sums, set_labels=labels[0:2])
plt.title('Aggression and Fear/Anxiety')
plt.show()

# Execute a chi-squared test of independence.
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('Chi-square Test of Independence:')
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)

### 8.2 Owner Directed Aggression and Fearful/Anxious Behavior:

In [ ]:
# Create the necessary dataframe.
fields = 'q03_main_1, q02_main_2'
labels = ['aggression', 'fear/anxiety']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
    
# Create a contingency table.
contingency = pd.crosstab(df['aggression'], df['fear/anxiety'])

# Convert the cross tabulated dataframe to a series of sums.
d = {'owner directed aggression': contingency[0][1],
     'fear/anxiety': contingency[1][0],
     'both': contingency[1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
labels = cross_sums.index.values
venn2(cross_sums, set_labels=labels[0:2])
plt.title('Owner Directed Aggression and Fear/Anxiety')
plt.show()

# Execute a chi-squared test of independence.
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('Chi-square Test of Independence:')
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)

### 8.3 Owner Directed Aggression and Separation Anxiety:

In [ ]:
# Create the necessary dataframe.
fields = 'q03_main_1, q04_9'
labels = ['aggression', 'anxiety']
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)
    
# Create a contingency table.
contingency = pd.crosstab(df['aggression'], df['anxiety'])

# Convert the cross tabulated dataframe to a series of sums.
d = {'owner directed aggression': contingency[0][1],
     'separation anxiety': contingency[1][0],
     'both': contingency[1][1]}   
cross_sums = pd.Series(d)

# Display the cross tabulated data as a venn diagram.
labels = cross_sums.index.values
venn2(cross_sums, set_labels=labels[0:2])
plt.title('Owner Direct Aggression and Separation Anxiety')
plt.show()

# Execute a chi-squared test of independence.
c, p, dof, expected = scs.chi2_contingency(contingency, correction=False)
print('Chi-square Test of Independence:')
print('chi2 = %f, p = %.2E, dof = %d' %(c, p, dof))
checkSignificance(p)

## 9 Impact of Gender and Neutered Status

### 9.1 On Study Prevalence:

This section examines the impact gender and neutered status had on the prevalence among all participating dogs (i.e. the probability of selecting a dog with the specific sex, neutered status, and behavior problem from the entire study population).

Note: \[I\] = intact, \[N\] = neutered

In [ ]:
# Create the necessary dataframe.
fields = []
labels = []
for counter, cat in enumerate(CATEGORY, 1):
    if counter > 10:
        counter += 1;
    fields.append('q02_main_{}'.format(counter))
    labels.append(cat)   
fields.extend(('dog_sex', 'dog_spayed'))
labels.extend(('Gender', 'Neutered'))
fields = ', '.join(fields)
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

df_intact_male = df[(df['Gender'] == 1) & (df['Neutered'] == 0)].drop(
    columns=['Gender', 'Neutered'])
df_neutered_male = df[(df['Gender'] == 1) & (df['Neutered'] == 1)].drop(
    columns=['Gender', 'Neutered'])
df_intact_female = df[(df['Gender'] == 2) & (df['Neutered'] == 0)].drop(
    columns=['Gender', 'Neutered'])
df_neutered_female = df[(df['Gender'] == 2) & (df['Neutered'] == 1)].drop(
    columns=['Gender', 'Neutered'])

intact_male_s = df_intact_male.sum()
neutered_male_s = df_neutered_male.sum()
intact_female_s = df_intact_female.sum()
neutered_female_s = df_neutered_female.sum()

intact_male_p = intact_male_s.apply(PREVALENCE)
neutered_male_p = neutered_male_s.apply(PREVALENCE)
intact_female_p = intact_female_s.apply(PREVALENCE)
neutered_female_p = neutered_female_s.apply(PREVALENCE)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 105, 115, 120],
                 header=dict(values=['Behavior Problem', 'Male [I] (%)',
                                     'Male [N] (%)', 'Female [I] (%)',
                                     'Female [N] (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 5),
                 cells=dict(values=[labels[:-2], intact_male_p.values,
                                    neutered_male_p.values, intact_female_p.values,
                                    neutered_female_p],
                            align = ['left'] * 5,
                            format = [None, '.2f' * 4]))]
layout = go.Layout(title='Impact of Gender and Neutered Status on Study Prevalence',
                   width=740,
                   height=400,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

### 9.2 On Group Prevalence:

This section looks at the impact gender and neutered status had on the prevalence among dogs of the same group (i.e. the probability of selecting a dog with the specific sex, neutered status, and behavior problem from the group of the study population that shares the same sex and neutered status).

Note: \[I\] = intact, \[N\] = neutered

In [ ]:
# Create the necessary dataframe.
fields = []
labels = []
for counter, cat in enumerate(CATEGORY, 1):
    if counter > 10:
        counter += 1;
    fields.append('q02_main_{}'.format(counter))
    labels.append(cat)   
fields.extend(('dog_sex', 'dog_spayed'))
labels.extend(('Gender', 'Neutered'))
fields = ', '.join(fields)
df = createNumericDataFrame(DOG_TABLE, fields, labels, filtered=True)

df_intact_male = df[(df['Gender'] == 1) & (df['Neutered'] == 0)].drop(
    columns=['Gender', 'Neutered'])
df_neutered_male = df[(df['Gender'] == 1) & (df['Neutered'] == 1)].drop(
    columns=['Gender', 'Neutered'])
df_intact_female = df[(df['Gender'] == 2) & (df['Neutered'] == 0)].drop(
    columns=['Gender', 'Neutered'])
df_neutered_female = df[(df['Gender'] == 2) & (df['Neutered'] == 1)].drop(
    columns=['Gender', 'Neutered'])

intact_male_c = len(df_intact_male.index)
neutered_male_c = len(df_neutered_male.index)
intact_female_c = len(df_intact_female.index)
neutered_female_c = len(df_neutered_female.index)

intact_male_s = df_intact_male.sum()
neutered_male_s = df_neutered_male.sum()
intact_female_s = df_intact_female.sum()
neutered_female_s = df_neutered_female.sum()

intact_male_p = intact_male_s.apply(lambda x: (x / intact_male_c) * 100)
neutered_male_p = neutered_male_s.apply(lambda x: (x / neutered_male_c) * 100)
intact_female_p = intact_female_s.apply(lambda x: (x / intact_female_c) * 100)
neutered_female_p = neutered_female_s.apply(lambda x: (x / neutered_female_c) * 100)

# Get individual behavior problem counts and display as a table.
data = [go.Table(columnwidth = [200, 100, 105, 115, 120],
                 header=dict(values=['Behavior Problem', 'Male [I] (%)',
                                     'Male [N] (%)', 'Female [I] (%)',
                                     'Female [N] (%)'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 5),
                 cells=dict(values=[labels[:-2], intact_male_p.values,
                                    neutered_male_p.values, intact_female_p.values,
                                    neutered_female_p],
                            align = ['left'] * 5,
                            format = [None, '.2f' * 4]))]
layout = go.Layout(title='Impact of Gender and Neutered Status on Group Prevalence',
                   width=740,
                   height=400,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
py.iplot(fig)

## 10 Chi-squared Matrix

### 10.1 Reference Chart:

In [ ]:
# Construct lists.
locs = []
labels = []
categories = []
for cat, subdict in FR.labels.items():
    for key, value in  subdict.items():
        locs.append(key)
        labels.append(value)
        categories.append(cat)

# Display lists in a chart for use as a reference.
data = [go.Table(columnwidth = [125, 225, 300],
                 header=dict(values=['Location', 'Category', 'Label'],
                             fill = dict(color='#B0C4DE'),
                             align = ['left'] * 3),
                 cells=dict(values=[locs, categories, labels],
                            align = ['left'] * 3))]
layout = go.Layout(width=650,
                   height=1600,
                   margin=go.Margin(l=50, r=50, b=50, t=50, pad=4))
fig = dict(data=data, layout=layout)
if SAVE_OUTPUT:
    py.plot(fig, image='svg', image_filename='heatmap_reference', image_height=1600,
            image_width=650)
py.iplot(fig)

### 10.2 Linear Heatmap:

In [ ]:
if CHI2_MATRIX.empty:
    CHI2_MATRIX = createChiSquaredMatrix()

# Heatmap
fig, ax = plt.subplots(figsize=(20, 15))
sns.heatmap(ax=ax, data=CHI2_MATRIX, cmap="PiYG", linewidths=.008, linecolor='black',
            cbar_kws={'label': 'chi-square values'})
plt.xticks(rotation=90)
plt.title('Linear Heatmap of Study Chi-squared Values', weight='bold')
if SAVE_OUTPUT:
    fig.savefig('../references/heatmap_linear', format='pdf')
plt.show()

### 10.3 Logarithmic Heatmap:

In [ ]:
if CHI2_MATRIX.empty:
    CHI2_MATRIX = createChiSquaredMatrix()

# Heatmap
from matplotlib.colors import LogNorm
from matplotlib import ticker
fig, ax = plt.subplots(figsize=(20, 15))
chi2_min = CHI2_MATRIX.min(skipna=True).min()
chi2_max = CHI2_MATRIX.max(skipna=True).max()
data = CHI2_MATRIX.fillna(1e-5)
cmap = sns.cubehelix_palette(light=1, as_cmap=True)
sns.heatmap(ax = ax, data=data, norm=LogNorm(vmin=chi2_min, vmax=chi2_max),
            cmap=cmap, linewidths=.008, linecolor='black',
            cbar_kws={'ticks': [0,1,10,1e2,1e3,1e4], 'label': 'chi-square values'})
plt.xticks(rotation=90)
plt.title('Logarithmic Heatmap of Study Chi-squared Values', weight='bold')
if SAVE_OUTPUT:
    fig.savefig('../references/heatmap_logarithmic', format='pdf')
plt.show()